In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import mwparserfromhell
import urllib.request
import json
import re
import numpy as np
import pickle
from tqdm.notebook import tqdm

In [52]:
with open('df.pickle', 'rb') as f:
    df = pickle.load(f)
# df = df.reset_index(drop=True)
df

,Name,Race,Affiliation
0,Terokk,Arakkoa,0
1,Talon_King_Ikiss,Arakkoa,Other
2,Archimonde,Man'ari eredar,Other
3,Kil'jaeden,Man'ari eredar,Other
4,Mannoroth,Annihilan,Other
...,...,...,...
2480,the_Jade_Warlord,Mogu,0
2481,the_Jailer,Eternal One,Other
2482,the_Monkey_King,Hozen,Other
2483,the_Stonewright,Venthyr,Other


In [48]:
# Clean text

from tqdm import tqdm
text_list = []
for name in tqdm(df['Name']):
    response = requests.get(
         'https://wowpedia.fandom.com/api.php',
             params={
             'action': 'query',
             'format': 'json',
             'titles': '{}'.format(name),
             'prop': 'revisions',
             'rvprop': 'content',
         }
     ).json()


    page = next(iter(response['query']['pages'].values()))
    try:
        wikicode = page['revisions'][0]['*']
        parsed_wikicode = mwparserfromhell.parse(wikicode).strip_code()
        
        if parsed_wikicode[:8].upper() == 'REDIRECT':
            t = parsed_wikicode[8:].strip().replace(" ", "_").split("\n")[0]
    
            response = requests.get(
                 'https://wowpedia.fandom.com/api.php',
                     params={
                     'action': 'query',
                     'format': 'json',
                     'titles': '{}'.format(t),
                     'prop': 'revisions',
                     'rvprop': 'content',
                 }).json()

            page = next(iter(response['query']['pages'].values()))
            wikicode2 = page['revisions'][0]['*']
            parsed_wikicode2 = mwparserfromhell.parse(wikicode2).strip_code()

            text_list.append(parsed_wikicode2)
        
        else:
            text_list.append(parsed_wikicode)
    except:
        text_list.append(" ")
    
# df.insert(2,'Clean Text',text_list)


100%|██████████████████████████████████████████████████████████████████████████████| 2485/2485 [17:04<00:00,  2.42it/s]


In [47]:
sas = "redirect Varian Wrynn\nCategory:Borean Tundra objects"
# sas = "redirect Master Gadrin"
print(sas[:8].upper())

print(sas[8:].strip().replace(" ", "_").split("\n")[0])


REDIRECT
Varian_Wrynn


In [51]:
#Find links for all characters:

baseurl = "https://wowpedia.fandom.com/api.php?"
action = "action=query"
title = "titles={}"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

WORD_RE = re.compile(r"\[{2}.*?\]{2}")

link_list=[]
for name in tqdm(list(df['Name'])):
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(name), dataformat)
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')

    links = []
    if "#REDIRECT" in wikitext:
        t = re.findall(r"\[{2}.*?\]{2}", wikitext)[0]
        t = t[2:-2].replace(" ", "_")
        query = "{}{}&{}&{}&{}".format(baseurl, action, content, title.format(t), dataformat)
        wikiresponse = urllib.request.urlopen(query)
        wikidata = wikiresponse.read()
        wikitext2 = wikidata.decode('utf-8')
 
        for word in WORD_RE.findall(wikitext2):
            word = word[2:-2].replace(" ", "_")
            if word in list(df['Name']) and word not in links:
                links.append(word)
        link_list.append(np.unique(links))

    else:
        for word in WORD_RE.findall(wikitext):
            word = word[2:-2].replace(" ", "_")
            if word in list(df['Name']) and word not in links:
                links.append(word)
        link_list.append(np.unique(links))
        


# df.insert(2,'Out Links',link_list)

100%|██████████████████████████████████████████████████████████████████████████████| 2485/2485 [18:20<00:00,  2.26it/s]


In [53]:
df.insert(3,'Clean Text',text_list)
df.insert(4,'Out Links',link_list)

In [54]:
df

,Name,Race,Affiliation,Clean Text,Out Links
0,Terokk,Arakkoa,0,Terokk was the greatest king of the arakkoa. T...,"[Anzu, Darkweaver_Syth, Hazzik, Isfar, Kirrik_..."
1,Talon_King_Ikiss,Arakkoa,Other,"Talon King Ikiss (pronounced Eye-kiss), the bo...","[Anzu, Darkweaver_Syth, Reshad, Sethe, Terokk]"
2,Archimonde,Man'ari eredar,Other,Archimonde (pronounced ARK-i-mond) was one of ...,"[Alexstrasza, Anetheron, Archimindy, Azgalor, ..."
3,Kil'jaeden,Man'ari eredar,Other,Kil'jaeden (sometimes spelled as Kil'Jaeden)Ar...,"[Akaari_Shadowgore, Archimonde, Arthas, Arthas..."
4,Mannoroth,Annihilan,Other,Mannoroth was the barbaric leader of the pit l...,"[Archimonde, Cenarius, Durotan, Garrosh, Garro..."
...,...,...,...,...,...
2480,the_Jade_Warlord,Mogu,0,The Jade Warlord was a powerful mogu spellcast...,"[Anduin_Wrynn, Monkey_King, Thunder_King]"
2481,the_Jailer,Eternal One,Other,"Zovaal, better known as the Jailer or the Bani...","[Anduin_Wrynn, Annhylde, Arbiter, Arthas_Menet..."
2482,the_Monkey_King,Hozen,Other,The Monkey King is a hozen who was a companion...,"[Anduin_Wrynn, Barnes, Emperor_Shaohao, Freya,..."
2483,the_Stonewright,Venthyr,Other,thumb|The Stonewright in World of Warcraft.\n\...,"[Denathrius, General_Draven, Huln_Highmountain..."


In [55]:
with open('df_text.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)